In [ ]:
# libraries 
import numpy as np
import pandas as pd
import requests
! pip install XlsxWriter
import xlsxwriter
import math


     |████████████████████████████████| 153kB 9.1MB/s 


In [ ]:
stocks = pd.read_csv("/content/sp_500_stocks.csv")
stocks.tail()

,Ticker
500,YUM
501,ZBH
502,ZBRA
503,ZION
504,ZTS


In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'



In [ ]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SQNMA)ET  AEAL T/CLDLNEGSB(AOKGRS', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'ifiolcaf', 'close': None, 'closeTime': None, 'closeSource': 'liofafic', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 129.88, 'latestSource': 'IEX Last Trade', 'latestTime': 'January 19, 2021', 'latestUpdate': 1672905832734, 'latestVolume': None, 'iexRealtimePrice': 127.82, 'iexRealtimeSize': 3, 'iexLastUpdated': 1675772382098, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 130.61, 'previousVolume': 112873823, 'change': 0.67, 'changePercent': 0.00521, 'volume': None, 'iexMarketPercent': 0.01172735463119475, 'iexVolume': 1056696, 'avgTotalVolume': 11071858

In [ ]:
price = data["latestPrice"]
market_cap = data["marketCap"]

In [ ]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
final_dataframe.append(
    pd.Series(
        [
          symbol,
         price,
         market_cap,
         'N/A'
        ], index = my_columns
    ), ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,129.88,2244761960302,N/A


In [ ]:
# Looping through the tickers in our list of stocks
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
  data  = requests.get(api_url).json()
  final_dataframe  = final_dataframe.append(
      pd.Series(
          [
           stock,
           data["latestPrice"],
           data["marketCap"],
           'N/A'
          ], index = my_columns
    ), ignore_index = True
)

In [ ]:
final_dataframe.tail()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
500,YUM,109.24,33717247904,N/A
501,ZBH,165.89,33853697058,N/A
502,ZBRA,411.33,22229414059,N/A
503,ZION,51.52,8390029231,N/A
504,ZTS,169.40,77502784583,N/A


In [ ]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))
  #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote,quote&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in string.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
             symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['quote']['marketCap'],
             'N/A'
            ], index = my_columns
        ), ignore_index = True
    )

In [ ]:
final_dataframe.tail()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
500,YUM,110.06,33740465848,N/A
501,ZBH,163.82,34287495681,N/A
502,ZBRA,413.28,22563798336,N/A
503,ZION,49.57,8364248064,N/A
504,ZTS,169.26,80641857164,N/A


In [ ]:
portfolio_size = input('Enter the value of your portfolio')

try: 